In [25]:
# from sklearn.feature_extraction.text import CountVectorizer
# import pandas as pd
import pdftotext
# import pandas as pd
# import os
# import unicodedata
import re
# import unicodedata
# from PyPDF2 import PdfFileReader
# from sklearn.feature_extraction.text import CountVectorizer
# import pandas as pd
# import pdftotext
# from collections import Counter
# import numpy as np
# from nltk.util import ngrams
from string import punctuation

In [26]:
def n_grams(text, n=2):
    '''
    wymagania: modul re
    text - artykul, na podstawie ktorego chcemy uzyskac liste n gramow
    zalozenie - 1.text zostal wczesniej poddany zabiegom lematyzacji i czyszczenia
                2.type(text)==str
    n>2, type(n)==int
    '''
    if type(text) != str:
        raise NameError("Zly typ argumentu text, text musi byc typu str")
    if type(n) != int or n<2:
        raise NameError("n musi byc liczba calkowita, n>2")
    
    
    text = text.lower()
    tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', text).split() #slowa z text zapisane do listy
    
    n_grams=list() #miejsce na n gramy
    
    for i in range(len(tokens)-n+1):
        s = tokens[i] 
        for j in range(1,n):
            s = s + " " + tokens[i+j]
        n_grams.append(s)
    
    return n_grams
    

In [27]:
def read_files(file_paths):
    text_list = []
    num_of_page = [0]*len(file_paths)
    it = 0
    for text in file_paths:
        with open(text, "rb") as f:
            pdf = pdftotext.PDF(f)
            el_of_list = ''
            #Ponieważ page in pdf  - to jest strona z artykułu to łącze stringi, pewnie to można lepiej
  
            for page in pdf:
                el_of_list = el_of_list+" startstrona "+ page
                num_of_page[it] += 1
            it+=1
                
            text_list.append(el_of_list)  
            
    return (text_list, num_of_page)

In [28]:
def normalize(file):
    #zmiana na małe litery
    file = file.lower()
    #Zmieniamy np. ó na o
    file = re.sub(r'[^\x20-\x7e]', '', file)
    #usuwanie cyfry
    file = re.sub(r'\d+', '', file)
    #usuwanie znaków specjalne
    file = ''.join(c for c in file if c not in punctuation)
    #Usuwamy  białe spacje
    file = re.sub(' +', ' ', file)
    return(file)

In [29]:
def remove_intr_refe(file):
    head, sep, result = file.partition("ntroductio")
    result, sep, tail = result.rpartition("eferences")
    return(result)

In [30]:
def remove_footer(file,page_number, n_gram=25):

    
    for i in list(range(n_gram, 2,-1)):
        main_dict = Counter(dict(filter(lambda x: x[1] >= page_number/2, Counter(n_grams(file,i)).items())))
        if (main_dict!=Counter() ):
            break
    
    #Usuwam nagłówki
    for i in list(main_dict.keys()):
        if(i in file):
            file = file.replace(i,"")
            
    return(file)
    


    

In [31]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
# from six import BytesIO as StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [32]:
import glob

In [64]:
path = '/home/marcin/Dane/'

files = [f for f in glob.glob(path + "*/*/*.pdf", recursive=True)]
files = files[0:100]

for f in files[0:3]:
    print(f)

/home/marcin/Dane/suzonowicz/Bahrain/AbuHijlehChakravartyGangulyLatifOsman.pdf
/home/marcin/Dane/suzonowicz/Bahrain/Alkhalifa.pdf
/home/marcin/Dane/suzonowicz/Bahrain/Akbaba.pdf


In [65]:
text2=[]
for file in files:
    try:
        t = read_files([file])[0][0]
    except:
        t = ""
        print("Błąd: " + file)
        
    text2.append(t)

In [66]:
files[5]

'/home/marcin/Dane/suzonowicz/Saudi Arabia/Bozyigit.pdf'

In [67]:
files[0]

'/home/marcin/Dane/suzonowicz/Bahrain/AbuHijlehChakravartyGangulyLatifOsman.pdf'

In [69]:
text=[]
i=0
for file in files:
    try:
        t = convert_pdf_to_txt(file)
    except:
        t = ""
        print("Błąd: " + str(i) + " - " + file)
    
    text.append(t)
    
    if(i%10==0):
        print(i)
    i = i+1    

if((i-1)%10!=0):
    print(i)
    

0
10
20
30
40
50
Błąd: 59 - /home/marcin/Dane/suzonowicz/Saudi Arabia/AlFawzanHoyMany.pdf
60
70
Błąd: 75 - /home/marcin/Dane/suzonowicz/Kuwait/AlDallalBoujarwahSaleh.pdf
80
90


In [70]:
convert_pdf_to_txt(files[4])[0:100]

'ResearchArticleFromMobiletoWearableSystem:AWearableRFIDSystemtoEnhanceTeachingandLearningConditionsS'

In [71]:
text2[4][0:100]

' startstrona Hindawi Publishing Corporation\nMobile Information Systems\nVolume 2016, Article ID 83649'

In [72]:
# W tym pliku nie zaczytały się spacje
files[4]

'/home/marcin/Dane/suzonowicz/Saudi Arabia/AbdulAlrazganBousbahiGhouzaliMarieSainte.pdf'

In [73]:
text2[4][0:200]

' startstrona Hindawi Publishing Corporation\nMobile Information Systems\nVolume 2016, Article ID 8364909, 10 pages\nhttp://dx.doi.org/10.1155/2016/8364909\nResearch Article\nFrom Mobile to Wearable System:'

In [74]:
text[4][0:200]

'ResearchArticleFromMobiletoWearableSystem:AWearableRFIDSystemtoEnhanceTeachingandLearningConditionsSouadLarabiMarie-Sainte,1MunaS.Alrazgan,1FatihaBousbahi,1SanaaGhouzali,1andWadoodAbdul21InformationTe'

In [93]:
l = []
for t in text:
    l.append(len(t))

In [134]:
text_clean = []

for i in range(len(text)):  
    t = text[i]
    if(len(t)>0):
        if(t.count("\n")/len(t)>0.3):
            t2 = t.replace("-\n", "").replace("\n\n", "\n").replace("\n", " ").replace("  ", "#").replace(" ", "").replace("#", " ")
    
        else:
            t2 = t.replace("-\n", "").replace("\n\n", "\n").replace("\n", " ")
    else:
        t2 = ""
        
    text_clean.append(t2)

In [111]:
for i in range(len(text_clean)):
    print("\n" + "############ " + str(i) + ". - " + files[i] + "############\n")
    print(text_clean[i][0:2000])
    


############ 0. - /home/marcin/Dane/suzonowicz/Bahrain/AbuHijlehChakravartyGangulyLatifOsman.pdf############

~) Pergamon Int. J. Math. Tools Manufact. Vol. 38, No. I2, pp. 7595, 1998 © 1997 Elsevier Science Ltd. All fights reserved Printed in Great Britain 08906955/98519.00 + .00 PII: S08906955(97)000084 STRETCHBENDING ANALYSIS OF USECTION BEAMS A. A. ELDOMIATYt:~ and A. A. ELSHARKAWYt§ (Received 1 July 1996; in final form 13 January 1997) AbstraetA complel~e analysis for stretehbending of Usection beams is introduced. The variations of the applied bending moment with the applied axial load for fully elastic, elasticplastic and fully plastic conditions are derived. The ultimate goal of such simulation technique is to develop a simple and fast technique to determine stresses, strains, springback and residual stresses as functions of parameters such as material characteristics, forming loads and the required geometry of Usection beams. The model proposed in the present study can be con